# Probar la búsqueda en Azure AI Search

To run this Notebook, you need to install Polyglot Notebooks for Visual Studio Code, which allows you to run .NET code in Jupyter Notebooks. You can find it here: [Polyglot Notebooks for Visual Studio Code](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode).

In [ ]:
// Instalar paquetes NuGet necesarios
#r "nuget: Azure.Search.Documents, *-*"
#r "nuget: Azure.Identity, *-*"
#r "nuget: DotNetEnv, 3.0.0"
#r "nuget: OpenAI, *-*"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Azure.Identity;
using Azure.Search.Documents;
using Azure.Search.Documents.Models;
using DotNetEnv;
using OpenAI;
using OpenAI.Embeddings;
using System.ClientModel;

// Cargar variables de entorno
Env.Load();
string endpoint = Environment.GetEnvironmentVariable("AZURE_AI_SEARCH_ENDPOINT");
string githubToken = Environment.GetEnvironmentVariable("GITHUB_TOKEN");
var credential = new DefaultAzureCredential();

const string INDEX_NAME = "my-index-net";

### Generar los embeddings para una consulta

In [ ]:
const string MODEL_NAME = "openai/text-embedding-3-small";
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint = new Uri("https://models.github.ai/inference")
};
EmbeddingClient embeddingClient = new(MODEL_NAME, new ApiKeyCredential(githubToken), openAIOptions);

string queryInput = "Resume la hoja de vida de Diego Zumárraga Mera";
OpenAIEmbeddingCollection embeddingResponse = embeddingClient.GenerateEmbeddings(new List<string> { queryInput });
var queryEmbedding = embeddingResponse.FirstOrDefault().ToFloats().ToArray();
Console.WriteLine($"Embedding length: {queryEmbedding.Length.ToString()}");

### Buscar la consulta en Azure AI Search

In [ ]:
var searchClient = new SearchClient(new Uri(endpoint), INDEX_NAME, credential);

var vectorQuery = new VectorizedQuery(queryEmbedding)
{
    Fields = { "embedding" },
    KNearestNeighborsCount = 3
};

var options = new SearchOptions()
{
    QueryType = SearchQueryType.Semantic,
    SemanticSearch = new()
                {
                    SemanticConfigurationName = "default",
                    QueryCaption = new(QueryCaptionType.Extractive),
                    QueryAnswer = new(QueryAnswerType.Extractive)
                },
    Size = 2
};

options.VectorSearch = new();
options.VectorSearch.Queries.Add(vectorQuery);

Console.WriteLine("Buscando resultados...");
var response = searchClient.Search<Dictionary<string, object>>(queryInput, options);

In [ ]:
string rawResults = response.GetRawResponse().Content.ToString();
Console.WriteLine($"Raw results: {rawResults}");

### Obtener la mejor respuesta

In [ ]:
var dicResults = System.Text.Json.JsonSerializer.Deserialize<Dictionary<string, object>>(rawResults);

object objAnswers; // List<Dictionary<string, object>>;
dicResults.TryGetValue("@search.answers", out objAnswers);

var answers = System.Text.Json.JsonSerializer.Deserialize<List<Dictionary<string, object>>>(objAnswers?.ToString() ?? string.Empty);
Console.WriteLine($"Number of answers: {answers?.Count ?? 0}");

if (answers != null && answers.Any())
{
    Console.WriteLine("Answers:");
    foreach (var answer in answers)
    {
        Console.WriteLine($"Text: {answer["text"]}");
        Console.WriteLine($"Score: {answer["score"]}");
    }
}

### Obtener todos los resultados

In [ ]:
var results = new List<Dictionary<string, object>>();
foreach (var doc in response.Value.GetResults())
{
    var d = doc.Document;
    results.Add(new Dictionary<string, object>
    {
        ["id"] = d["id"],
        ["title"] = d["title"],
        ["content"] = d["content"],
        ["rerankerScore"] = d.ContainsKey("@search.rerankerScore") ? d["@search.rerankerScore"] : null
    });
}

Console.WriteLine($"Found {results.Count} results");
for (int i = 0; i < results.Count; i++)
{
    var doc = results[i];
    Console.WriteLine($"Result {i + 1}:");
    Console.WriteLine($"Title: {doc["title"]}");
    Console.WriteLine($"Content: {doc["content"]}");
    Console.WriteLine($"Reranker Score: {doc["rerankerScore"]}");
    Console.WriteLine();
}